# Задание 3.1 - Сверточные нейронные сети (Convolutional Neural Networks)

Это последнее задание на numpy, вы до него дожили! Остался последний марш-бросок, дальше только PyTorch.

В этом задании вы реализуете свою собственную сверточную нейронную сеть.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
# %env CUPY_ACCELERATORS=cutensor

In [3]:
from dataset import load_svhn, random_split_train_val
from layers import FullyConnectedLayer, ReLULayer, ConvolutionalLayer, MaxPoolingLayer, Flattener
from model import ConvNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD, Adam
from metrics import multiclass_accuracy
import cupy as cp

/home/michael/jup_env/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/michael/jup_env/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [4]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train[0].shape

(28, 28)

In [6]:
from sklearn.model_selection import train_test_split
def prepare_for_neural_network(train_X, test_X):
    train_X = train_X.astype(np.float) / 255.0
    test_X = test_X.astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_X, axis = 0)
    train_X -= mean_image
    test_X -= mean_image
    return train_X, test_X
    
train_X, test_X = prepare_for_neural_network(x_train, x_test)
# Split train into train and val
train_X, val_X, train_y, val_y = train_test_split(train_X, y_train, random_state=42, test_size=0.3)
test_y = y_test

<ipython-input-6-b52e92a3a2f9>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_X = train_X.astype(np.float) / 255.0
<ipython-input-6-b52e92a3a2f9>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_X = test_X.astype(np.float) / 255.0


In [7]:
train_X = train_X.reshape((-1, 28, 28, 1))
val_X = val_X.reshape((-1, 28, 28, 1))
test_X = test_X.reshape((-1, 28, 28, 1))

In [8]:
train_X.shape, val_X.shape, test_X.shape

((42000, 28, 28, 1), (18000, 28, 28, 1), (10000, 28, 28, 1))

# TRAIN

In [ ]:
batch_size = 2048
train_size = (42000 // batch_size)*batch_size
val_size = (18000 // batch_size)*batch_size

model = ConvNet()
dataset = Dataset(train_X[:train_size], train_y[:train_size], val_X[:val_size], val_y[:val_size])

trainer = Trainer(model, dataset, Adam(), learning_rate=0.001, num_epochs=100, batch_size=batch_size)

loss_history, train_history, val_history = trainer.fit()



Epoch 1  Train loss: 1.88720, Train accuracy: 0.24404, Val loss: 1.83434, val accuracy: 0.38641, time: 6.70
New best val loss: 1.8343403549217763



Epoch 2  Train loss: 0.95079, Train accuracy: 0.53528, Val loss: 0.95184, val accuracy: 0.66516, time: 6.23
New best val loss: 0.9518408009364332



Epoch 3  Train loss: 0.67656, Train accuracy: 0.72605, Val loss: 0.62223, val accuracy: 0.79297, time: 6.22
New best val loss: 0.6222269302601494



Epoch 4  Train loss: 0.48201, Train accuracy: 0.82229, Val loss: 0.46409, val accuracy: 0.85187, time: 6.22
New best val loss: 0.4640885287600858



Epoch 5  Train loss: 0.37230, Train accuracy: 0.87200, Val loss: 0.37929, val accuracy: 0.88367, time: 6.22
New best val loss: 0.3792856040234377


...


Epoch 11  Train loss: 0.16547, Train accuracy: 0.94543, Val loss: 0.19887, val accuracy: 0.93774, time: 6.30
New best val loss: 0.19887148023627205



Epoch 12  Train loss: 0.18327, Train accuracy: 0.94915, Val loss: 0.19135, val accuracy: 0.93976, time: 6.18
New best val loss: 0.19135062549514148



Epoch 13  Train loss: 0.15147, Train accuracy: 0.95283, Val loss: 0.18624, val accuracy: 0.94208, time: 6.35
New best val loss: 0.18623832048682487



Epoch 14  Train loss: 0.13004, Train accuracy: 0.95637, Val loss: 0.18132, val accuracy: 0.94415, time: 6.42
New best val loss: 0.18132435991385715



Epoch 15  Train loss: 0.14031, Train accuracy: 0.95928, Val loss: 0.16836, val accuracy: 0.94855, time: 6.20
New best val loss: 0.16835563919070357


In [ ]:
# del model

In [ ]:
plt.plot(train_history)
plt.plot(val_history)

In [ ]:
plt.plot(loss_history)

In [9]:
best_model = ConvNet()
best_model.load_params('best_model')

In [10]:
batch_size = 1000
test_size = (10000 // batch_size)*batch_size

best_model.predict_accuracy(test_X[:test_size], test_y[:test_size], batch_size)

100%|██████████| 10/10 [00:23<00:00,  2.34s/it]


0.9501